### Using a decorator to prime a coroutine

We always have to prime a coroutine before using it
- very repetitive
- pattern is same every time:

- g = gen() -> creates coroutine instance
- next(g) or g.send(None) -> primes coroutine

This is a perfect example of where you can use a decorator to do this work for us!

#### Creating a function to auto prime coroutines

In [1]:
def prime(gen_fn):
    g = gen_fn() # creates the generator
    next(g) # primes the generator
    return g # returns primed generator

So if we had a generator function like the one below...

In [2]:
def echo():
    while True:
        received = yield
        print(received)

We could now call prime with echo like so:

In [3]:
echo_gen = prime(echo)

We would now be able to call send directly...

In [4]:
echo_gen.send('hello')

hello


#### A decorator approach

We still have to remember to call the *prime* function for our echo coroutine before we can use it

Since echo is a coroutine, we know we always ahve to prime it first

So let's write a decorator that will repalce our generator function with another function that will automatically prime it when we create an instance of it

In [5]:
def coroutine(gen_f):
    def prime():
        g = gen_fn()
        next(g)
        return g
    return prime

So now we can decorate the echo function using the coroutine decorator!

In [6]:
@coroutine
def echo():
    while True:
        received = yield
        print(received)

#### Expanding the decorator

In the above example, you cannot pass arguments to the generator function

In [7]:
def coroutine(gen_fn):
    def prime(*args, **kwargs):
        g = gen_fn(*args, **kwargs)
        next(g)
        return g
    return prime

We can now use this coroutine function to prime **ANY** generator function that will act like a coroutine

#### Code Examples

In [8]:
def coroutine(gen_fn):
    def inner():
        gen = gen_fn()
        next(gen)
        return gen
    return inner

In [19]:
@coroutine
def echo():
    while True:
        received = yield
        print(received)

In [20]:
e = echo()

In [21]:
from inspect import getgeneratorstate

In [22]:
getgeneratorstate(e)

'GEN_SUSPENDED'

In [23]:
e.send('hello')

hello


Now lets add more so that we can handle fn arguments

In [24]:
def coroutine(gen_fn):
    def inner(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        next(gen)
        return gen
    return inner

In [25]:
import math

@coroutine
def power_up(p):
    result = None
    while True:
        received = yield result
        result = math.pow(received, p)

In [26]:
squares = power_up(2)
cubes = power_up(3)

In [27]:
squares.send(2)

4.0

In [28]:
cubes.send(2)

8.0

In [29]:
squares.send('abc')

TypeError: must be real number, not str

In [30]:
getgeneratorstate(squares)

'GEN_CLOSED'

In [31]:
@coroutine
def power_up(p):
    result = None
    while True:
        received = yield result
        try:
            result = math.pow(received, p)
        except TypeError:
            result = None

In [32]:
squares = power_up(2)

In [33]:
squares.send(2)

4.0

In [34]:
squares.send('abc')

In [35]:
getgeneratorstate(squares)

'GEN_SUSPENDED'

In [36]:
squares.send(3)

9.0

In [37]:
squares.close()

In [38]:
getgeneratorstate(squares)

'GEN_CLOSED'